构造一组输入数据X和其对应的标签y

In [1]:
import numpy

numpy里面**reshape**使用

In [2]:

x_values=[i for i in range(11)]
print(x_values)
x_train=numpy.array(x_values,dtype=numpy.float32)
print(x_train)
print(x_train.shape)
x_train=x_train.reshape(-1,1)
print(x_train)
x_train.shape

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
(11,)
[[ 0.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]]


(11, 1)

In [3]:

y_values=[2*i+1 for i in x_values]
print(y_values)
y_train=numpy.array(y_values,dtype=numpy.float32)
print(y_train)
print(y_train.shape)
y_train=y_train.reshape(-1,1)
print(y_train)
y_train.shape

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
[ 1.  3.  5.  7.  9. 11. 13. 15. 17. 19. 21.]
(11,)
[[ 1.]
 [ 3.]
 [ 5.]
 [ 7.]
 [ 9.]
 [11.]
 [13.]
 [15.]
 [17.]
 [19.]
 [21.]]


(11, 1)

In [4]:
import torch
import torch.nn as nn

**线性回归模型**
* 子类定义了构造函数父类的构造函数不会调用，如果要调用父类构造函数  需要使用super的方法调用
* 线性回归是一个不加激活函数的全连接层

In [5]:
class LinearRegressionMode(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(LinearRegressionMode,self).__init__()
        self.linear=nn.Linear(input_dim,output_dim)
    def forward(self,x):
        out=self.linear(x)
        return out

In [6]:
input_dim=1
output_dim=1
model=LinearRegressionMode(input_dim,output_dim)
model

LinearRegressionMode(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

learning rate：
* 学习率较小时，收敛到极值的速度较慢。
* 学习率较大时，容易在搜索过程中发生震荡。

In [7]:
#训练轮数
epochs=1000
learning_rate=0.01
# SGD 随机梯度下降优化器
optimizer =torch.optim.SGD(model.parameters(),lr=learning_rate)
criterion=nn.MSELoss()

训练模型

In [8]:
for epoch in range(epochs):
    epoch +=1
    #将数据转化为tensor格式
    inputs=torch.from_numpy(x_train)
    labels=torch.from_numpy(y_train)
    
    #梯度需要清零每一次迭代 否则会累加
    optimizer.zero_grad()
    
    #向前传播
    outputs=model(inputs)
    
    #计算损失
    loss=criterion(outputs,labels)
    
    #反向传播
    loss.backward()
    
    #更新权重参数
    optimizer.step()
    if epoch%100==0:
        print("epoch{},loss{}".format(epoch,loss.item()))
    

epoch100,loss0.056658338755369186
epoch200,loss0.01843164674937725
epoch300,loss0.005996090359985828
epoch400,loss0.001950611942447722
epoch500,loss0.0006345593137666583
epoch600,loss0.00020643332391045988
epoch700,loss6.715828203596175e-05
epoch800,loss2.1848474716534838e-05
epoch900,loss7.107049441401614e-06
epoch1000,loss2.312574451934779e-06


使用GPU训练
* 把数据与模型传到cuda里面

In [9]:
# 1.模型传入
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device)
model.to(device)
# 2.损失函数传入
criterion.to(device)
# 3.数据传入
inputs=torch.from_numpy(x_train).to(device)
labels=torch.from_numpy(y_train).to(device)

for epoch in range(epochs):
    epoch +=1
    
    #梯度需要清零每一次迭代 否则会累加
    optimizer.zero_grad()
    
    #向前传播
    outputs=model(inputs)
    
    #计算损失
    loss=criterion(outputs,labels)
    
    #反向传播
    loss.backward()
    
    #更新权重参数
    optimizer.step()
    if epoch%100==0:
        print("epoch{},loss{}".format(epoch,loss.item()))

epoch100,loss7.519403197875363e-07
epoch200,loss2.4467618686685455e-07
epoch300,loss7.959179271210814e-08
epoch400,loss2.5915053214475847e-08
epoch500,loss8.417626595758065e-09
epoch600,loss2.7453437301261374e-09
epoch700,loss9.067562256603878e-10
epoch800,loss3.0156757646615517e-10
epoch900,loss1.0099041047872959e-10
epoch1000,loss4.062237698998494e-11


模型的保存与读取

In [10]:
#保存
torch.save(model.state_dict(),'model.pth')

In [11]:
#读取
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [12]:
model(torch.from_numpy(x_train).to(device))

tensor([[ 1.0000],
        [ 3.0000],
        [ 5.0000],
        [ 7.0000],
        [ 9.0000],
        [11.0000],
        [13.0000],
        [15.0000],
        [17.0000],
        [19.0000],
        [21.0000]], device='cuda:0', grad_fn=<AddmmBackward0>)